# Data Collection

### Objective

Collect financial and economic data for UK public companies to build a corporate distress early warning system. We combine company-level financial statements with relevant economic indicators to create a comprehensive dataset for risk analysis.

### Data Sources

| Source | Purpose | Coverage |
|--------|---------|----------|
| **Yahoo Finance** | Company financials (balance sheet, P&L, cash flow), stock prices, market data | FTSE listed companies, 4-5 years history |
| **ONS API** | UK economic indicators by sector and region (unemployment, wages, inflation) | UK-wide with regional breakdowns |
| **FRED API** | Interest rates, commodity prices, global economic indicators | Historical time series data |



In [25]:
# Data manipulation
import pandas as pd
import numpy as np

# API and web requests
import requests
import yfinance as yf

# Date and time handling
from datetime import datetime, timedelta
import time

# Database
import sqlite3
from sqlalchemy import create_engine

# Environment and configuration
import os
from dotenv import load_dotenv

# Utilities
import json
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Progress tracking
from tqdm import tqdm

# Load environment variables
load_dotenv()

print("All libraries loaded successfully")
print(f"Pandas version: {pd.__version__}")
print(f"YFinance version: {yf.__version__}")

All libraries loaded successfully
Pandas version: 2.2.2
YFinance version: 0.2.54


In [15]:
import yfinance as yf
import pandas as pd

# Test 5 UK companies
test_tickers = {
    'TSCO.L': 'Tesco',
    'SGE.L': 'Sage Group', 
    'OCDO.L': 'Ocado',
    'NXT.L': 'Next',
    'RR.L': 'Rolls-Royce'
}

results = []

for ticker, name in test_tickers.items():
    stock = yf.Ticker(ticker)
    
    # Check what's available
    has_bs = stock.balance_sheet is not None and not stock.balance_sheet.empty
    has_income = stock.financials is not None and not stock.financials.empty
    has_cf = stock.cashflow is not None and not stock.cashflow.empty
    
    results.append({
        'Company': name,
        'Ticker': ticker,
        'Balance Sheet': has_bs,
        'Income Statement': has_income,
        'Cash Flow': has_cf,
        'BS Rows': len(stock.balance_sheet) if has_bs else 0,
        'Income Rows': len(stock.financials) if has_income else 0
    })

# Display results
df = pd.DataFrame(results)
print(df.to_string(index=False))

print("\n" + "="*50)
print("\nSample Tesco Balance Sheet items:")
if not test_tickers:
    tesco = yf.Ticker('TSCO.L')
    if tesco.balance_sheet is not None:
        print(tesco.balance_sheet.index.tolist()[:15])

    Company Ticker  Balance Sheet  Income Statement  Cash Flow  BS Rows  Income Rows
      Tesco TSCO.L           True              True       True       78           51
 Sage Group  SGE.L           True              True       True       67           48
      Ocado OCDO.L           True              True       True       80           50
       Next  NXT.L           True              True       True       77           49
Rolls-Royce   RR.L           True              True       True       85           49


Sample Tesco Balance Sheet items:


In [16]:
# Check actual financial metrics available
tesco = yf.Ticker('TSCO.L')

print("Balance Sheet metrics (first 20):")
print(tesco.balance_sheet.index.tolist()[:20])

print("\n" + "="*50)
print("\nIncome Statement metrics (first 20):")
print(tesco.financials.index.tolist()[:20])

print("\n" + "="*50)
print("\nCash Flow metrics (first 15):")
print(tesco.cashflow.index.tolist()[:15])

print("\n" + "="*50)
print("\nSample data structure (Tesco Balance Sheet - latest 2 years):")
print(tesco.balance_sheet.iloc[:10, :2])

Balance Sheet metrics (first 20):
['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued', 'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital', 'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations', 'Common Stock Equity', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Minority Interest', 'Stockholders Equity', 'Treasury Stock', 'Retained Earnings', 'Additional Paid In Capital', 'Capital Stock', 'Common Stock']


Income Statement metrics (first 20):
['Tax Effect Of Unusual Items', 'Tax Rate For Calcs', 'Normalized EBITDA', 'Total Unusual Items', 'Total Unusual Items Excluding Goodwill', 'Net Income From Continuing Operation Net Minority Interest', 'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA', 'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income', 'Normalized Income', 'Net Income From Continuing And Discontinued Operation', 'Total Expenses', 'Total Operating Income As Reported', 'Diluted Ave

In [17]:
# Verify we have multi-year data for all
for ticker, name in test_tickers.items():
    stock = yf.Ticker(ticker)
    years = stock.balance_sheet.columns.tolist()
    print(f"{name}: {len(years)} years of data | Latest: {years[0].strftime('%Y-%m')}")

Tesco: 5 years of data | Latest: 2025-02
Sage Group: 4 years of data | Latest: 2024-09
Ocado: 5 years of data | Latest: 2024-11
Next: 5 years of data | Latest: 2025-01
Rolls-Royce: 5 years of data | Latest: 2024-12


### Validation Results

Tested 5 FTSE companies - all have comprehensive financial data:
- Tesco: 5 years | Sage: 4 years | Ocado: 5 years | Next: 5 years | Rolls-Royce: 5 years
- All companies have 67-85 balance sheet line items
- Income statements: 48-51 metrics
- Cash flow statements available

**Conclusion:** Yahoo Finance provides sufficient structured data for analysis.

In [63]:
# Expanded FTSE company universe - 50 companies across sectors
companies = [
    # Consumer Goods & Retail
    ('TSCO.L', 'Tesco', 'Consumer Goods'),
    ('MKS.L', 'Marks & Spencer', 'Consumer Goods'),
    ('NXT.L', 'Next', 'Consumer Goods'),
    ('SBRY.L', 'Sainsbury', 'Consumer Goods'),
    ('OCDO.L', 'Ocado', 'Consumer Goods'),
    ('MRW.L', 'Morrison', 'Consumer Goods'),
    ('KGF.L', 'Kingfisher', 'Consumer Goods'),
    ('JD.L', 'JD Sports', 'Consumer Goods'),
    ('FRAS.L', 'Frasers Group', 'Consumer Goods'),
    ('ABF.L', 'Associated British Foods', 'Consumer Goods'),
    
    # Technology & Software
    ('SGE.L', 'Sage Group', 'Technology'),
    ('AUTO.L', 'Auto Trader', 'Technology'),
    ('MNDI.L', 'Mondi', 'Technology'),
    ('SMIN.L', 'Smiths Group', 'Technology'),
    ('HLMA.L', 'Halma', 'Technology'),
    
    # Industrials & Aerospace
    ('RR.L', 'Rolls-Royce', 'Industrials'),
    ('BA.L', 'BAE Systems', 'Industrials'),
    ('MGGT.L', 'Meggitt', 'Industrials'),
    ('WTB.L', 'Whitbread', 'Industrials'),
    ('IMB.L', 'Imperial Brands', 'Industrials'),
    ('BNZL.L', 'Bunzl', 'Industrials'),
    ('RMV.L', 'Rightmove', 'Industrials'),
    ('SMDS.L', 'DS Smith', 'Industrials'),
    
    # Energy & Utilities
    ('BP.L', 'BP', 'Energy'),
    ('SHEL.L', 'Shell', 'Energy'),
    ('SSE.L', 'SSE', 'Energy'),
    ('NG.L', 'National Grid', 'Energy'),
    ('ULVR.L', 'Unilever', 'Energy'),
    
    # Healthcare & Pharmaceuticals
    ('AZN.L', 'AstraZeneca', 'Healthcare'),
    ('GSK.L', 'GSK', 'Healthcare'),
    ('DGE.L', 'Diageo', 'Healthcare'),
    ('RKT.L', 'Reckitt Benckiser', 'Healthcare'),
    ('HIK.L', 'Hikma Pharmaceuticals', 'Healthcare'),
    
    # Financials (non-banks)
    ('LSEG.L', 'London Stock Exchange', 'Financials'),
    ('III.L', '3i Group', 'Financials'),
    ('FCIT.L', 'F&C Investment Trust', 'Financials'),
    ('PSN.L', 'Persimmon', 'Financials'),
    ('BDEV.L', 'Barratt Developments', 'Financials'),
    
    # Telecommunications & Media
    ('VOD.L', 'Vodafone', 'Telecommunications'),
    ('BT-A.L', 'BT Group', 'Telecommunications'),
    ('ITV.L', 'ITV', 'Telecommunications'),
    ('WPP.L', 'WPP', 'Telecommunications'),
    
    # Materials & Mining
    ('AAL.L', 'Anglo American', 'Materials'),
    ('GLEN.L', 'Glencore', 'Materials'),
    ('RIO.L', 'Rio Tinto', 'Materials'),
    ('ANTO.L', 'Antofagasta', 'Materials'),
    
    # Consumer Services
    ('BATS.L', 'British American Tobacco', 'Consumer Services'),
    ('CCH.L', 'Coca-Cola HBC', 'Consumer Services'),
    ('CPG.L', 'Compass Group', 'Consumer Services'),


# Additional FTSE 250 & Small Cap
    ('MCRO.L', 'Micro Focus', 'Technology'),
    ('RCP.L', 'RPC Group', 'Industrials'),
    ('GNS.L', 'Genus', 'Healthcare'),
    ('UTG.L', 'Unite Group', 'Financials'),
    ('BME.L', 'B&M European', 'Consumer Goods'),
    ('DLN.L', 'Derwent London', 'Financials'),
    ('BKG.L', 'Berkeley Group', 'Financials'),
    ('SHED.L', 'Sheds', 'Consumer Goods'),
    ('WOS.L', 'Workspace Group', 'Financials'),
    ('PNN.L', 'Pennon', 'Energy'),
    ('TSCO.L', 'Tesco', 'Consumer Goods'),
    ('BWY.L', 'Bellway', 'Financials'),
    ('LGEN.L', 'Legal & General', 'Financials'),
    ('AV.L', 'Aviva', 'Financials'),
    ('PRU.L', 'Prudential', 'Financials'),
    ('DLG.L', 'Direct Line', 'Financials'),
    ('RSA.L', 'RSA Insurance', 'Financials'),
    ('AGR.L', 'Assura', 'Healthcare'),
    ('DRUM.L', 'Diploma', 'Industrials'),
    ('FEVR.L', 'Fevertree', 'Consumer Goods'),
    ('CAKE.L', 'Finsbury Food', 'Consumer Goods'),
    ('MGNS.L', 'Morgan Sindall', 'Industrials'),
    ('BOOT.L', 'Boots', 'Healthcare'),
    ('BBY.L', 'Balfour Beatty', 'Industrials'),
    ('CBRY.L', 'Cadbury', 'Consumer Goods'),
    ('CURY.L', 'Currys', 'Consumer Goods'),
    ('GRG.L', 'Greggs', 'Consumer Services'),
    ('JDW.L', 'Wetherspoon', 'Consumer Services'),
    ('MARS.L', 'Marstons', 'Consumer Services'),
    ('MCB.L', 'McColl', 'Consumer Goods'),
    ('MTO.L', 'Mitie', 'Industrials'),
    ('NWG.L', 'Natwest', 'Financials'),
    ('LLOY.L', 'Lloyds Banking', 'Financials'),
    ('BARC.L', 'Barclays', 'Financials'),
    ('HSBA.L', 'HSBC', 'Financials'),
    ('RBS.L', 'RBS', 'Financials'),
    ('CBG.L', 'Close Brothers', 'Financials'),
    ('JLEN.L', 'JLEN Environmental', 'Energy'),
    ('NESF.L', 'Nestle', 'Consumer Goods'),
    ('PDL.L', 'Pets at Home', 'Consumer Services'),
    ('ASHM.L', 'Ashmore', 'Financials'),
    ('CWK.L', 'Cranswick', 'Consumer Goods'),
    ('SOLG.L', 'Solgold', 'Materials'),
    ('AHT.L', 'Ashtead', 'Industrials'),
    ('SMT.L', 'Scottish Mortgage', 'Financials'),
    ('SYNC.L', 'Syncona', 'Healthcare'),
    ('GCP.L', 'GCP Infrastructure', 'Financials'),
    ('VMUK.L', 'Virgin Money', 'Financials'),
    ('OSB.L', 'OneSavings Bank', 'Financials'),
    ('WIZZ.L', 'Wizz Air', 'Consumer Services'),
    ('EZJ.L', 'EasyJet', 'Consumer Services'),
    ('IAG.L', 'IAG', 'Consumer Services'),
    ('TUI.L', 'TUI', 'Consumer Services'),
    ('SSL.L', 'Subsea 7', 'Energy'),
    ('TATE.L', 'Tate & Lyle', 'Consumer Goods'),
    ('DFS.L', 'DFS Furniture', 'Consumer Goods'),
    ('FOUR.L', '4imprint', 'Industrials'),
    ('GAW.L', 'Games Workshop', 'Consumer Goods'),
    ('FDM.L', 'FDM Group', 'Technology'),
    ('CAPC.L', 'Capco', 'Financials'),
    ('TSTL.L', 'Tristel', 'Healthcare'),
    ('SCIN.L', 'ScS', 'Consumer Goods'),
    ('RWS.L', 'RWS Holdings', 'Technology'),
    ('PTEC.L', 'Playtech', 'Technology'),
    ('SLA.L', 'Standard Life Aberdeen', 'Financials'),
    ('PCTN.L', 'Picton Property', 'Financials'),
    ('IHG.L', 'InterContinental Hotels', 'Consumer Services'),
    ('SBRY.L', 'Sainsburys', 'Consumer Goods'),
    ('WHR.L', 'Wetherspoons', 'Consumer Services'),
    ('DPLM.L', 'Diploma', 'Industrials'),
    ('RDT.L', 'Redde Northgate', 'Industrials'),
    ('SIG.L', 'SIG', 'Industrials'),
    ('TEP.L', 'Telecom Plus', 'Telecommunications'),
    ('TRY.L', 'Troy Income', 'Financials'),
    ('VEC.L', 'Vectura', 'Healthcare'),
    ('VCT.L', 'Victrex', 'Materials'),
    ('VSVS.L', 'Vesuvius', 'Industrials'),
    ('WIZZ.L', 'Wizz Air', 'Consumer Services'),
    ('WKP.L', 'Workspace', 'Financials'),
    ('XPP.L', 'XP Power', 'Technology'),
    ('YU.L', 'Yu Group', 'Energy'),
    ('ZTF.L', 'Z', 'Financials'),
    ('ASC.L', 'Asos', 'Consumer Goods'),
    ('BOO.L', 'Boohoo', 'Consumer Goods'),
    ('JET2.L', 'Jet2', 'Consumer Services'),
    ('INDV.L', 'Indivior', 'Healthcare'),
    ('ICG.L', 'Intermediate Capital', 'Financials'),
    ('IGG.L', 'IG Group', 'Financials'),
    ('HFD.L', 'Halfords', 'Consumer Goods'),
    ('HAS.L', 'Hays', 'Industrials'),
    ('GPOR.L', 'Great Portland', 'Financials'),
    ('GOG.L', 'Go-Ahead', 'Industrials'),
    ('FOXT.L', 'Foxtons', 'Financials'),
    ('FOOT.L', 'Footasylum', 'Consumer Goods'),
    ('FDEV.L', 'FD Technologies', 'Technology'),
    ('ESNT.L', 'Essentra', 'Industrials'),
    ('ENQ.L', 'Enquest', 'Energy'),
    ('ENT.L', 'Enterprise Inns', 'Consumer Services'),
    ('DJAN.L', 'Djan', 'Technology'),
    ('DEB.L', 'Debenhams', 'Consumer Goods'),
    ('CTEC.L', 'ConvaTec', 'Healthcare'),
    ('COA.L', 'Coats', 'Consumer Goods'),
    ('CLIG.L', 'Clinigen', 'Healthcare'),
    ('CLLN.L', 'Clarkson', 'Industrials'),
    ('CMCX.L', 'CMC Markets', 'Financials'),
    ('CAR.L', 'Carillion', 'Industrials'),
    ('BUSH.L', 'Bush', 'Industrials'),
    ('BBOX.L', 'Tritax Big Box', 'Financials'),
    ('BBA.L', 'BBA Aviation', 'Industrials'),
    ('ARTA.L', 'Artisan Real Estate', 'Financials'),
    ('AML.L', 'Aston Martin', 'Consumer Goods'),
    ('ALFA.L', 'Alpha Financial', 'Financials'),
    ('AAIF.L', 'Aberdeen Asian Income', 'Financials'),
    ('888.L', '888 Holdings', 'Consumer Services'),
    ('STAN.L', 'Standard Chartered', 'Financials'),
    ('SGRO.L', 'Segro', 'Financials'),
    ('SMWH.L', 'WH Smith', 'Consumer Goods'),
    ('SPT.L', 'Spirent', 'Technology'),
    ('SSP.L', 'SSP Group', 'Consumer Services'),
    ('SXS.L', 'Spectris', 'Technology'),
    ('TCG.L', 'Thomas Cook', 'Consumer Services'),
    ('TEM.L', 'Templeton', 'Financials'),
    ('TMPL.L', 'Temple Bar', 'Financials'),
    ('TPK.L', 'Travis Perkins', 'Industrials'),
    ('TRN.L', 'Trainline', 'Technology'),
    ('TPX.L', 'Topps Tiles', 'Consumer Goods'),
    ('ULE.L', 'Ultra Electronics', 'Technology'),
    ('ULS.L', 'ULS Technology', 'Technology'),
    ('UKW.L', 'UK Water', 'Energy'),
    ('UKCM.L', 'UK Commercial Property', 'Financials'),
    ('UDP.L', 'UDP', 'Industrials'),
    ('VED.L', 'Vedanta', 'Materials'),
    ('VTU.L', 'Vitec', 'Technology'),
    ('WOR.L', 'Workspace', 'Financials'),
    ('WIZZ.L', 'Wizz Air', 'Consumer Services'),
    ('WIX.L', 'Wix', 'Technology'),
    ('WTI.L', 'Witan', 'Financials'),
    ('WSP.L', 'WSP Global', 'Industrials'),
    ('ZPG.L', 'Zoopla', 'Technology'),
]

df_universe = pd.DataFrame(companies, columns=['Ticker', 'Company', 'Sector'])
print(f"Company universe: {len(df_universe)} companies")
print(f"\nSector breakdown:")
print(df_universe['Sector'].value_counts())

Company universe: 188 companies

Sector breakdown:
Sector
Financials            48
Consumer Goods        34
Industrials           29
Technology            20
Consumer Services     20
Healthcare            14
Energy                11
Materials              7
Telecommunications     5
Name: count, dtype: int64


In [64]:
# Collect data for all companies
print(f"Collecting data for {len(df_universe)} companies...\n")
print("This will take approximately 2 minutes...\n")

all_company_data = {}
failed_collections = []

for idx, row in df_universe.iterrows():
    ticker = row['Ticker']
    company = row['Company']
    
    print(f"[{idx+1}/{len(df_universe)}] {company} ({ticker})...", end=" ")
    
    data = collect_company_financials(ticker)
    
    if data['success']:
        all_company_data[ticker] = data
        print("✓")
    else:
        failed_collections.append({'ticker': ticker, 'company': company, 'error': data['error']})
        print(f"✗")
    
    time.sleep(0.5)  # Rate limiting

print(f"\n{'='*50}")
print(f"Successfully collected: {len(all_company_data)}/{len(df_universe)}")
print(f"Success rate: {len(all_company_data)/len(df_universe)*100:.1f}%")
print(f"Failed: {len(failed_collections)}")


This will take approximately 2 minutes...

[1/188] Tesco (TSCO.L)... ✓
[2/188] Marks & Spencer (MKS.L)... ✓
[3/188] Next (NXT.L)... ✓
[4/188] Sainsbury (SBRY.L)... ✓
[5/188] Ocado (OCDO.L)... ✓
[6/188] Morrison (MRW.L)... ✓
[7/188] Kingfisher (KGF.L)... ✓
[8/188] JD Sports (JD.L)... ✓
[9/188] Frasers Group (FRAS.L)... ✓
[10/188] Associated British Foods (ABF.L)... ✓
[11/188] Sage Group (SGE.L)... ✓
[12/188] Auto Trader (AUTO.L)... ✓
[13/188] Mondi (MNDI.L)... ✓
[14/188] Smiths Group (SMIN.L)... ✓
[15/188] Halma (HLMA.L)... ✓
[16/188] Rolls-Royce (RR.L)... ✓
[17/188] BAE Systems (BA.L)... ✓
[18/188] Meggitt (MGGT.L)... ✓
[19/188] Whitbread (WTB.L)... ✓
[20/188] Imperial Brands (IMB.L)... ✓
[21/188] Bunzl (BNZL.L)... ✓
[22/188] Rightmove (RMV.L)... ✓
[23/188] DS Smith (SMDS.L)... ✗
[24/188] BP (BP.L)... ✓
[25/188] Shell (SHEL.L)... ✓
[26/188] SSE (SSE.L)... ✓
[27/188] National Grid (NG.L)... ✓
[28/188] Unilever (ULVR.L)... ✓
[29/188] AstraZeneca (AZN.L)... ✓
[30/188] GSK (GSK.L)... ✓
[3

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BUSH.L?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BUSH.L&crumb=Dtb5odR0Th4


✗
[157/188] Tritax Big Box (BBOX.L)... ✓
[158/188] BBA Aviation (BBA.L)... ✗
[159/188] Artisan Real Estate (ARTA.L)... ✓
[160/188] Aston Martin (AML.L)... ✓
[161/188] Alpha Financial (ALFA.L)... ✓
[162/188] Aberdeen Asian Income (AAIF.L)... ✓
[163/188] 888 Holdings (888.L)... ✗
[164/188] Standard Chartered (STAN.L)... ✓
[165/188] Segro (SGRO.L)... ✓
[166/188] WH Smith (SMWH.L)... ✓
[167/188] Spirent (SPT.L)... ✗
[168/188] SSP Group (SSP.L)... ✓
[169/188] Spectris (SXS.L)... ✓
[170/188] Thomas Cook (TCG.L)... ✗
[171/188] Templeton (TEM.L)... ✓
[172/188] Temple Bar (TMPL.L)... ✓
[173/188] Travis Perkins (TPK.L)... ✓
[174/188] Trainline (TRN.L)... ✓
[175/188] Topps Tiles (TPX.L)... ✓
[176/188] Ultra Electronics (ULE.L)... ✓
[177/188] ULS Technology (ULS.L)... ✓
[178/188] UK Water (UKW.L)... ✓
[179/188] UK Commercial Property (UKCM.L)... ✗
[180/188] UDP (UDP.L)... 

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/UDP.L?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=UDP.L&crumb=Dtb5odR0Th4


✗
[181/188] Vedanta (VED.L)... ✓
[182/188] Vitec (VTU.L)... ✓
[183/188] Workspace (WOR.L)... ✓
[184/188] Wizz Air (WIZZ.L)... ✓
[185/188] Wix (WIX.L)... ✓
[186/188] Witan (WTI.L)... ✓
[187/188] WSP Global (WSP.L)... ✓
[188/188] Zoopla (ZPG.L)... ✓

Successfully collected: 165/188
Success rate: 87.8%
Failed: 19


### Financial Data Collection

Building collection function to extract all financial statements from Yahoo Finance. Testing on single company before batch processing.

In [65]:
# Extract and normalize financial data
financial_data = []

for ticker, data in all_company_data.items():
    company_name = df_universe[df_universe['Ticker'] == ticker]['Company'].values[0]
    sector = df_universe[df_universe['Ticker'] == ticker]['Sector'].values[0]
    
    bs = data['balance_sheet']
    income = data['income_statement']
    cf = data['cash_flow']
    
    for date in bs.columns:
        record = {
            'ticker': ticker,
            'company': company_name,
            'sector': sector,
            'date': date,
            'total_debt': bs.loc['Total Debt', date] if 'Total Debt' in bs.index else None,
            'cash': bs.loc['Cash And Cash Equivalents', date] if 'Cash And Cash Equivalents' in bs.index else None,
            'total_assets': bs.loc['Total Assets', date] if 'Total Assets' in bs.index else None,
            'stockholders_equity': bs.loc['Stockholders Equity', date] if 'Stockholders Equity' in bs.index else None,
            'revenue': income.loc['Total Revenue', date] if date in income.columns and 'Total Revenue' in income.index else None,
            'operating_income': income.loc['Operating Income', date] if date in income.columns and 'Operating Income' in income.index else None,
            'net_income': income.loc['Net Income', date] if date in income.columns and 'Net Income' in income.index else None,
            'operating_cash_flow': cf.loc['Operating Cash Flow', date] if date in cf.columns and 'Operating Cash Flow' in cf.index else None,
            'free_cash_flow': cf.loc['Free Cash Flow', date] if date in cf.columns and 'Free Cash Flow' in cf.index else None
        }
        financial_data.append(record)

df_financials = pd.DataFrame(financial_data)
df_financials['date'] = pd.to_datetime(df_financials['date'])
df_financials = df_financials.sort_values(['ticker', 'date'])

print(f"Total financial records: {len(df_financials)}")
print(f"Date range: {df_financials['date'].min().date()} to {df_financials['date'].max().date()}")
print(f"Average records per company: {len(df_financials)/165:.1f}")
print(f"\nCompanies with 4+ years data: {(df_financials.groupby('ticker').size() >= 4).sum()}")
print(f"\nSample data:")
print(df_financials[['company', 'date', 'total_debt', 'revenue', 'net_income']].head(10))

Total financial records: 667
Date range: 2020-11-30 to 2025-08-31
Average records per company: 4.0

Companies with 4+ years data: 139

Sample data:
                      company       date    total_debt       revenue  \
604     Aberdeen Asian Income 2021-12-31  4.675300e+07  4.917800e+07   
603     Aberdeen Asian Income 2022-12-31  4.096700e+07 -1.482400e+07   
602     Aberdeen Asian Income 2023-12-31  3.212300e+07  1.124700e+07   
601     Aberdeen Asian Income 2024-12-31  3.242200e+07  3.757000e+07   
190            Anglo American 2020-12-31           NaN           NaN   
189            Anglo American 2021-12-31  1.285600e+10  4.155400e+10   
188            Anglo American 2022-12-31  1.436500e+10  3.511800e+10   
187            Anglo American 2023-12-31  1.691200e+10  3.065200e+10   
186            Anglo American 2024-12-31  1.821000e+10  2.729000e+10   
47   Associated British Foods 2021-09-30           NaN           NaN   

       net_income  
604  4.531100e+07  
603 -1.706600e+07  

In [66]:
# Calculate financial ratios
df_financials['net_debt'] = df_financials['total_debt'] - df_financials['cash']
df_financials['debt_to_equity'] = df_financials['total_debt'] / df_financials['stockholders_equity']
df_financials['operating_margin'] = df_financials['operating_income'] / df_financials['revenue']
df_financials['net_margin'] = df_financials['net_income'] / df_financials['revenue']
df_financials['cash_to_assets'] = df_financials['cash'] / df_financials['total_assets']

# Calculate YoY growth
df_financials = df_financials.sort_values(['ticker', 'date'])
df_financials['revenue_growth'] = df_financials.groupby('ticker')['revenue'].pct_change()
df_financials['income_growth'] = df_financials.groupby('ticker')['net_income'].pct_change()

# Flag distress
df_financials['negative_equity'] = df_financials['stockholders_equity'] < 0
df_financials['in_distress'] = (
    (df_financials['stockholders_equity'] < 0) | 
    (df_financials['debt_to_equity'] > 3)
)

print("Financial metrics calculated")
print(f"\nData completeness:")
key_metrics = ['total_debt', 'revenue', 'net_income', 'debt_to_equity', 'operating_margin']
for metric in key_metrics:
    pct = (1 - df_financials[metric].isna().sum() / len(df_financials)) * 100
    print(f"{metric:20s}: {pct:5.1f}%")

print(f"\nDistress identification:")
print(f"Records with negative equity: {df_financials['negative_equity'].sum()}")
print(f"Records flagged as distress: {df_financials['in_distress'].sum()}")
print(f"Healthy records for analysis: {(~df_financials['in_distress']).sum()}")

Financial metrics calculated

Data completeness:
total_debt          :  81.0%
revenue             :  83.4%
net_income          :  83.4%
debt_to_equity      :  81.0%
operating_margin    :  67.2%

Distress identification:
Records with negative equity: 10
Records flagged as distress: 38
Healthy records for analysis: 629


### Data Structuring

- Extracting key financial metrics into normalized format with one record per company-year. 

- Focusing on core health indicators: debt, cash, profitability, and cash flow.

In [50]:
# Extract and normalize financial data into structured format
financial_data = []

for ticker, data in all_company_data.items():
    company_name = df_universe[df_universe['Ticker'] == ticker]['Company'].values[0]
    sector = df_universe[df_universe['Ticker'] == ticker]['Sector'].values[0]
    
    # Get balance sheet data
    bs = data['balance_sheet']
    income = data['income_statement']
    cf = data['cash_flow']
    
    # For each year, create a record
    for date in bs.columns:
        record = {
            'ticker': ticker,
            'company': company_name,
            'sector': sector,
            'date': date,
            # Balance sheet metrics
            'total_debt': bs.loc['Total Debt', date] if 'Total Debt' in bs.index else None,
            'cash': bs.loc['Cash And Cash Equivalents', date] if 'Cash And Cash Equivalents' in bs.index else None,
            'total_assets': bs.loc['Total Assets', date] if 'Total Assets' in bs.index else None,
            'stockholders_equity': bs.loc['Stockholders Equity', date] if 'Stockholders Equity' in bs.index else None,
            # Income statement metrics
            'revenue': income.loc['Total Revenue', date] if date in income.columns and 'Total Revenue' in income.index else None,
            'operating_income': income.loc['Operating Income', date] if date in income.columns and 'Operating Income' in income.index else None,
            'net_income': income.loc['Net Income', date] if date in income.columns and 'Net Income' in income.index else None,
            # Cash flow metrics
            'operating_cash_flow': cf.loc['Operating Cash Flow', date] if date in cf.columns and 'Operating Cash Flow' in cf.index else None,
            'free_cash_flow': cf.loc['Free Cash Flow', date] if date in cf.columns and 'Free Cash Flow' in cf.index else None
        }
        financial_data.append(record)

df_financials = pd.DataFrame(financial_data)
df_financials['date'] = pd.to_datetime(df_financials['date'])
df_financials = df_financials.sort_values(['ticker', 'date'])

print(f"Structured financial records: {len(df_financials)}")
print(f"Date range: {df_financials['date'].min()} to {df_financials['date'].max()}")
print(f"\nSample data:")
print(df_financials[['company', 'date', 'total_debt', 'revenue', 'net_income']].head(10))

Structured financial records: 218
Date range: 2020-11-30 00:00:00 to 2025-08-31 00:00:00

Sample data:
                      company       date    total_debt       revenue  \
190            Anglo American 2020-12-31           NaN           NaN   
189            Anglo American 2021-12-31  1.285600e+10  4.155400e+10   
188            Anglo American 2022-12-31  1.436500e+10  3.511800e+10   
187            Anglo American 2023-12-31  1.691200e+10  3.065200e+10   
186            Anglo American 2024-12-31  1.821000e+10  2.729000e+10   
47   Associated British Foods 2021-09-30           NaN           NaN   
46   Associated British Foods 2022-08-31           NaN           NaN   
45   Associated British Foods 2022-09-30           NaN           NaN   
44   Associated British Foods 2023-08-31  3.722000e+09  1.975000e+10   
43   Associated British Foods 2023-09-30  3.722000e+09  1.975000e+10   

       net_income  
190           NaN  
189  8.562000e+09  
188  4.514000e+09  
187  2.830000e+08  
186 

### Financial Ratio Calculation

Computing standard financial health metrics:
- Leverage ratios (debt-to-equity, net debt)
- Profitability margins (operating, net)
- Liquidity indicators (cash-to-assets)
- Growth rates (revenue, income YoY)

These ratios normalize company size and enable cross-company comparison.

In [51]:
# Calculate key financial ratios and health indicators
df_financials['net_debt'] = df_financials['total_debt'] - df_financials['cash']
df_financials['debt_to_equity'] = df_financials['total_debt'] / df_financials['stockholders_equity']
df_financials['operating_margin'] = df_financials['operating_income'] / df_financials['revenue']
df_financials['net_margin'] = df_financials['net_income'] / df_financials['revenue']
df_financials['cash_to_assets'] = df_financials['cash'] / df_financials['total_assets']

# Calculate year-over-year growth rates
df_financials = df_financials.sort_values(['ticker', 'date'])
df_financials['revenue_growth'] = df_financials.groupby('ticker')['revenue'].pct_change()
df_financials['income_growth'] = df_financials.groupby('ticker')['net_income'].pct_change()

# Summary statistics
print("Financial metrics calculated")
print(f"\nSummary statistics:")
print(df_financials[['debt_to_equity', 'operating_margin', 'net_margin', 'revenue_growth']].describe())

print(f"\nMissing values by column:")
print(df_financials.isnull().sum()[df_financials.isnull().sum() > 0])

Financial metrics calculated

Summary statistics:
       debt_to_equity  operating_margin  net_margin  revenue_growth
count      176.000000        172.000000  180.000000      135.000000
mean         0.814158          0.161611    0.144850        0.030919
std          0.784977          0.161972    0.229363        0.280881
min         -5.630482         -0.389657   -0.526592       -2.164257
25%          0.427725          0.071784    0.042467       -0.014007
50%          0.866547          0.141724    0.086627        0.045680
75%          1.246152          0.211807    0.148862        0.117965
max          2.203449          0.741589    1.034848        0.844157

Missing values by column:
total_debt             42
cash                   38
total_assets           38
stockholders_equity    38
revenue                38
operating_income       46
net_income             38
operating_cash_flow    42
free_cash_flow         38
net_debt               42
debt_to_equity         42
operating_margin       46

### Economic Data Collection

Collecting macroeconomic indicators from ONS and FRED to provide context for company performance analysis.

In [67]:
# ONS Data Collection
def get_ons_data(series_id: str, dataset: str) -> pd.DataFrame:
    """Collect ONS time series data."""
    url = f"https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/{series_id}/{dataset}/data"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    if response.status_code == 200:
        data = response.json()
        records = []
        for entry in data.get('years', []) + data.get('quarters', []) + data.get('months', []):
            records.append({
                'period': entry.get('date'),
                'value': float(entry.get('value')) if entry.get('value') else None,
                'label': entry.get('label')
            })
        return pd.DataFrame(records)
    else:
        raise Exception(f"Failed: {response.status_code}")

# Parse CPI dates
def parse_cpi_date(period_str):
    try:
        parts = period_str.strip().split()
        if len(parts) == 2:
            year, month = parts
            month_map = {'JAN':'01','FEB':'02','MAR':'03','APR':'04','MAY':'05','JUN':'06',
                        'JUL':'07','AUG':'08','SEP':'09','OCT':'10','NOV':'11','DEC':'12'}
            if month in month_map:
                return pd.to_datetime(f"{year}-{month_map[month]}-01")
    except:
        pass
    return None

print("Collecting economic data...\n")

# CPI
df_cpi = get_ons_data('d7g7', 'mm23')
df_cpi['date'] = df_cpi['period'].apply(parse_cpi_date)
df_cpi_clean = df_cpi[df_cpi['date'].notna()].copy()
df_cpi_monthly = df_cpi_clean[['date', 'value']].rename(columns={'value': 'cpi'})

print(f"✓ CPI: {len(df_cpi_monthly)} records")




✓ CPI: 442 records


In [69]:
# Check CPI coverage for our financial data period
df_cpi_relevant = df_cpi_monthly[df_cpi_monthly['date'] >= '2019-01-01']
print(f"CPI records since 2019: {len(df_cpi_relevant)}")
print(f"Date range: {df_cpi_relevant['date'].min()} to {df_cpi_relevant['date'].max()}")
print(f"\nLatest CPI values:")
print(df_cpi_relevant.tail())

CPI records since 2019: 82
Date range: 2019-01-01 00:00:00 to 2025-10-01 00:00:00

Latest CPI values:
          date  cpi
620 2025-06-01  3.6
621 2025-07-01  3.8
622 2025-08-01  3.8
623 2025-09-01  3.8
624 2025-10-01  3.6


In [68]:
# FRED data
from fredapi import Fred
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

df_uk_rate = fred.get_series('IRLTLT01GBM156N', observation_start='2019-01-01').to_frame('uk_bond_yield').reset_index()
df_uk_rate.columns = ['date', 'uk_bond_yield']

df_oil = fred.get_series('DCOILBRENTEU', observation_start='2019-01-01').to_frame('brent_oil').reset_index()
df_oil.columns = ['date', 'brent_oil']

df_gdp = fred.get_series('CPMNACSCAB1GQUK', observation_start='2019-01-01').to_frame('uk_gdp').reset_index()
df_gdp.columns = ['date', 'uk_gdp']
df_gdp['uk_gdp_growth'] = df_gdp['uk_gdp'].pct_change() * 100

print(f"✓ UK bond yields: {len(df_uk_rate)} records")
print(f"✓ Brent oil: {len(df_oil)} records")
print(f"✓ UK GDP: {len(df_gdp)} records")

print("\nEconomic data collection complete.")

✓ UK bond yields: 82 records
✓ Brent oil: 1795 records
✓ UK GDP: 7 records

Economic data collection complete.


In [70]:
# Verify GDP growth calculation
print("GDP data check:")
print(df_gdp[['date', 'uk_gdp', 'uk_gdp_growth']].tail())

GDP data check:
        date    uk_gdp  uk_gdp_growth
2 2019-07-01  558286.0       1.145365
3 2019-10-01  559387.0       0.197211
4 2020-01-01  554315.0      -0.906707
5 2020-04-01  476341.0     -14.066731
6 2020-07-01  540164.0      13.398595


## Data Storage

Storing complete dataset in SQLite database for analysis in subsequent notebooks.

**Final Dataset:**
- 165 companies across 9 sectors
- 667 company-year financial records
- 629 healthy records, 38 distressed
- 5 years of economic indicators

In [71]:
# Create database
db_path = '../data/processed/company_data.db'
os.makedirs(os.path.dirname(db_path), exist_ok=True)
engine = create_engine(f'sqlite:///{db_path}')

print("Storing data in SQLite database...\n")

# Store all tables
df_financials.to_sql('financials', engine, if_exists='replace', index=False)
print(f"✓ Stored financials: {len(df_financials)} records")

df_universe.to_sql('companies', engine, if_exists='replace', index=False)
print(f"✓ Stored company list: {len(df_universe)} companies")

df_cpi_monthly.to_sql('cpi', engine, if_exists='replace', index=False)
print(f"✓ Stored CPI: {len(df_cpi_monthly)} records")

df_uk_rate.to_sql('uk_bond_yields', engine, if_exists='replace', index=False)
print(f"✓ Stored bond yields: {len(df_uk_rate)} records")

df_oil.to_sql('oil_prices', engine, if_exists='replace', index=False)
print(f"✓ Stored oil prices: {len(df_oil)} records")

df_gdp.to_sql('gdp', engine, if_exists='replace', index=False)
print(f"✓ Stored GDP: {len(df_gdp)} records")

print(f"\n{'='*50}")
print(f"Database saved: {db_path}")
print(f"Total tables: 6")
print(f"\nDataset summary:")
print(f"- Companies: {len(df_universe)}")
print(f"- Financial records: {len(df_financials)}")
print(f"- Healthy records: {(~df_financials['in_distress']).sum()}")
print(f"- Distressed records: {df_financials['in_distress'].sum()}")
print(f"- Date range: {df_financials['date'].min().date()} to {df_financials['date'].max().date()}")

Storing data in SQLite database...

✓ Stored financials: 667 records
✓ Stored company list: 188 companies
✓ Stored CPI: 442 records
✓ Stored bond yields: 82 records
✓ Stored oil prices: 1795 records
✓ Stored GDP: 7 records

Database saved: ../data/processed/company_data.db
Total tables: 6

Dataset summary:
- Companies: 188
- Financial records: 667
- Healthy records: 629
- Distressed records: 38
- Date range: 2020-11-30 to 2025-08-31


### **Summary**

Successfully collected comprehensive dataset for UK corporate distress analysis:

**Company Data:**
- 188 FTSE companies across 9 sectors
- 667 company-year financial records (2020-2025)
- 629 healthy records, 38 distressed cases
- Average 4 years of data per company
- 81-83% data completeness on key metrics

**Economic Data:**
- CPI inflation: 442 records (monthly)
- UK bond yields: 82 records (monthly)
- Brent oil: 1,795 records (daily)
- UK GDP growth: 7 records (quarterly)

**Data Quality:**
- 87.8% successful collection rate
- Clean financial ratios calculated
- Distress flags identified (Rolls-Royce + 37 others)
- Ready for exploratory analysis and ML modeling

**Next Steps:** Exploratory Data Analysis and Visualisations in the next dataset

In [72]:
# Check what metadata we captured
test_ticker = 'TSCO.L'
if test_ticker in all_company_data:
    test_company = all_company_data[test_ticker]
    
    print("Available info keys (first 30):")
    info_keys = list(test_company['info'].keys())
    print(info_keys[:30])
    
    print("\n" + "="*50)
    print("\nKey metadata we can extract:")
    useful_fields = ['marketCap', 'sector', 'industry', 'fullTimeEmployees', 
                     'country', 'website', 'longBusinessSummary']
    
    for field in useful_fields:
        value = test_company['info'].get(field, 'N/A')
        if field == 'longBusinessSummary' and value != 'N/A':
            print(f"{field}: {value[:100]}...")  # Truncate long text
        else:
            print(f"{field}: {value}")

Available info keys (first 30):
['address1', 'address2', 'city', 'zip', 'country', 'phone', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'governanceEpochDate', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh']


Key metadata we can extract:
marketCap: 29101740032
sector: Consumer Defensive
industry: Grocery Stores
fullTimeEmployees: 340000
country: United Kingdom
website: https://www.tescoplc.com
longBusinessSummary: Tesco PLC, together with its subsidiaries, operates as a grocery retailer in the United Kingdom, Rep...
